In [1]:
import os
import numpy as np

## Positive Expected Value Betting Routine

Project idea / design:

Want: A general routine for finding the EV and Kelly Criterion bet amount for a given boosted bet. 

Input: 
    1. given odds of boosted event. 
    2. given odds of each subevent
    3. given odds of each complement subevent
    
Alternate input (WIP): if we don't have the given odds of each subevent and complement subevent, we will develop an ML model to predict the VIG for an event, given event odds and event type (moneyline, spread, etc.)
    
F1: Take inputs (2,3) and calculate the vig of each subevent. 
F2: Use vig from F1 to find best estimate of real probability for each subevent and complement subevent. 
F3: Calculate probability of the boosted (main) event. 
F4: Use Kelly criterion to determine betting amount. 


For example: We have three subevents, showing given odds that three different NBA players score 30+ points in a game on a given night. 
P1: -174	+130
P2: -320	+225
P3: -190	+152

In [10]:
num_sub_events = int(float(input("How many subevents do you have? ")))

How many subevents do you have? 3


In [11]:
print(f"The boosted event has a total of {num_sub_events} subevents.")

The boosted event has a total of 3 subevents.


In [13]:
arr = input("Input tuples of subevent odds and complement subevent odds. ") 
#for example, if a given subevent has odds +130, complement odds -140, input: 120 -140
l = list(map(int,arr.split(' '))) 

Input tuples of subevent odds and complement subevent odds. -174 130 -320 225 -190 152


In [21]:
subevent_array = np.array(l).reshape(num_sub_events, 2)
vig = np.empty((num_sub_events, 1))
subevent_complement_prob = np.empty((num_sub_events, 2))

In [39]:
subevent_array

array([[-174,  130],
       [-320,  225],
       [-190,  152]])

In [41]:
def calculate_vig(subevent_array): 
    for i in range(0, num_sub_events):
        if subevent_array[i][0] > 0: # 1st column, i.e. subevent odds
            subevent_complement_prob[i][0] = 100/(subevent_array[i][0]+100)
        else: 
            subevent_complement_prob[i][0] = abs(subevent_array[i][0]) / (abs(subevent_array[i][0]) + 100)
        if subevent_array[i][1] > 0: #2nd column, i.e. complement subevent odds
            subevent_complement_prob[i][1] = 100/(subevent_array[i][1]+100)
        else: 
            subevent_complement_prob[i][1] = abs(subevent_array[i][1]) / (abs(subevent_array[i][1]) + 100)
        vig[i][0] =  subevent_complement_prob[i][0]+ subevent_complement_prob[i][1]-1
    
    return vig, subevent_complement_prob
    
    

In [45]:
vig, subevent_complement_prob = calculate_vig(subevent_array)

In [62]:
print(vig)
print(subevent_complement_prob)

[[0.06981911]
 [0.06959707]
 [0.05199781]]
[[0.6350365  0.43478261]
 [0.76190476 0.30769231]
 [0.65517241 0.3968254 ]]


In [59]:
def calculate_actual_probs(subevent_complement_prob, vig):
    actual_probs = np.empty((num_sub_events, 2))
    for i in range(num_sub_events):
        actual_probs[i] = np.divide(subevent_complement_prob[i],1+vig[i])
    return actual_probs

In [65]:
actual_probs = calculate_actual_probs(subevent_complement_prob, vig)

If you want to do a simple parlay of the subevents, follow the below cell. 

In [68]:
def intersect_subevent_prob(actual_probs):
    """
   Input the matrix with shape (number of sub events, 2) that contains true probabilities for each event. 
    Assumes the subevents are independent. 
    """
    intersection_prob = np.empty((1,2))
    intersection_prob = np.prod(actual_probs, axis = 0)
    return intersection_prob[0]
    

In [70]:
parlay_all_subevents_prob = intersect_subevent_prob(actual_probs)

In [73]:
print(f"The true probability of all subevents occurring simultaneously is {np.round(parlay_all_subevents_prob*100,1)}%. ")

The true probability of all subevents occurring simultaneously is 26.3%. 


In [ ]:
def calculate_fair_odds(true_probability):
    """
    Input: true_probability should be a float between 0 and 1.
    Output: Fair odds on the usual betting scale (100+ representing unlikely events, -100- representing likely events)
    """
    if true_probability <= .5:
        true_odds = 100/true_probability -100
    elif true_probability > .5 and true_probability <= 1:
        true_odds = -100/(1-true_probability)+100
    else:
        "Invalid input probability"
    return true_odds

In [75]:
def union_subevent_prob(actual_sub_probs):
    """
    Input a column vector of probabilities to take the union of. At most 2 probabilities. Uses the inclusion exclusion formula. 
    Output probability of inclusive or. 
    """
    
  

IndentationError: expected an indented block (3382153900.py, line 13)

In [85]:
actual_sub_probs

[[1.0], [0.5], [0.3333333333333333]]

In [74]:
actual_sub_probs[0]*actual_sub_probs[i] for i in range(1, len(actual_sub_probs))

SyntaxError: invalid syntax (3356352350.py, line 1)

In [67]:
len(actual_sub_probs)

3